<a href="https://colab.research.google.com/github/ozakiryota/cart_pole/blob/main/cartpole_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cart-Pole


Install OpenAI Gym (https://gym.openai.com)

In [3]:
!pip install gym

Install the packages for visualizing Gym

In [4]:
!apt update
!apt install xvfb
!pip install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

## DQN

Import Gym

In [5]:
import gym

Import the packages for visualizing Gym

In [6]:
import base64
import io
from gym.wrappers import Monitor
from IPython import display
from pyvirtualdisplay import Display

Import required packages

In [7]:
import numpy as np

import torch
from torch import nn
import torch.optim as optim

Net class

In [8]:
class Net(nn.Module):
    def __init__(self, num_states, dim_mid, num_actions):
        super().__init__()

        self.fc = nn.Sequential(
            nn.Linear(num_states, dim_mid),
            nn.ReLU(),
            nn.Linear(dim_mid, dim_mid),
            nn.ReLU(),
            nn.Linear(dim_mid, num_actions)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

Brain class

In [9]:
class Brain:
    def __init__(self, num_states, num_actions, gamma, r, lr):
        self.num_states = num_states
        self.num_actions = num_actions

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print("self.device = ", self.device)
        self.net = Net(num_states, 32, num_actions)
        self.net.to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.RMSprop(self.net.parameters(), lr=0.01)

        self.eps = 1.0  # for epsilon greedy algorithm
        self.gamma = gamma
        self.r = r
        self.lr = lr
 
    def updateQnet(self, obs_numpy, action, reward, next_obs_numpy):
        obs_tensor = torch.from_numpy(obs_numpy).float()
        obs_tensor.unsqueeze_(0)
        obs_tensor = obs_tensor.to(self.device)

        next_obs_tensor = torch.from_numpy(next_obs_numpy).float()
        next_obs_tensor.unsqueeze_(0)
        next_obs_tensor = next_obs_tensor.to(self.device)

        self.net.train()
        q = self.net(obs_tensor)

        with torch.no_grad():
            self.net.eval()
            next_q = self.net(next_obs_tensor)

            labels = q.clone()
            labels[:, action] = reward + self.gamma*np.max(next_q.cpu().detach().numpy(), axis=1)[0]
        
        loss = self.criterion(q, labels)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def getAction(self, obs_numpy, is_training):
        if is_training and np.random.rand() < self.eps:
            action = np.random.randint(self.num_actions)
        else:
            obs_tensor = torch.from_numpy(obs_numpy).float()
            obs_tensor.unsqueeze_(0)
            obs_tensor = obs_tensor.to(self.device)
            with torch.no_grad():
                self.net.eval()
                q = self.net(obs_tensor)
                action = np.argmax(q.cpu().detach().numpy(), axis=1)[0]
        ## update eps
        if is_training and self.eps > 0.1:
            self.eps *= self.r
        return action

Agent class

In [10]:
class Agent:
    def __init__(self, num_states, num_actions, gamma, r, lr):
        self.brain = Brain(num_states, num_actions, gamma, r, lr)
 
    def updateQnet(self, obs, action, reward, next_obs):
        self.brain.updateQnet(obs, action, reward, next_obs)
 
    def getAction(self, obs, is_training):
        action = self.brain.getAction(obs, is_training)
        return action

Environment class

In [11]:
class Environment:
    def __init__(self, num_episodes, max_step, gamma, r, lr):
        ## parameters
        self.num_episodes = num_episodes
        self.max_step = max_step
        ## environment
        self.env = Monitor(gym.make('CartPole-v0'), './videos/', force=True)
        ## agent
        num_states = self.env.observation_space.shape[0]    # position, velocity, angle, angular velocity
        num_actions = self.env.action_space.n
        self.agent = Agent(num_states, num_actions, gamma, r, lr)

    def train(self):
        num_completed_episodes = 0
        
        for episode in range(self.num_episodes):
            obs = self.env.reset()
            episode_reward = 0
 
            for step in range(self.max_step):
                ## get action
                action = self.agent.getAction(obs, is_training=True)
                ## observe next step
                next_obs, _, is_done, _ = self.env.step(action)
                ## get reward
                if is_done:
                    if step < max_step - 1:
                        reward = -100
                    else:
                        reward = 1
                        num_completed_episodes += 1
                else:
                    reward = 1
                episode_reward += reward
                ## update
                self.agent.updateQnet(obs, action, reward, next_obs)
                ## to next step
                obs = next_obs

                if is_done:
                    print('{0} Episode: Finished after {1} time steps with reward {2}'.format(episode, step+1, episode_reward))
                    break
        print("num_completed_episodes = ", num_completed_episodes)

    def evaluate(self):
        obs = self.env.reset()
        
        for step in range(self.max_step):
            ## get action
            action = self.agent.getAction(obs, is_training=False)
            ## observe next step
            next_obs, _, is_done, _ = self.env.step(action)
            ## to next step
            obs = next_obs

            if is_done:
                print('Evaluation: Finished after {} time steps'.format(step+1))
                break

Prepare showing videos of the restults

In [14]:
def show_video(env):
    env.reset()
    for frame in env.videos:
        print("frame = ", frame)
        video = io.open(frame[0], 'r+b').read()
        encoded = base64.b64encode(video)

        display.display(display.HTML(data="""
            <video alt="test" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>
            """.format(encoded.decode('ascii')))
        )

Run training

In [15]:
## display
virtual_display = Display()
virtual_display.start()

## parameters
num_episodes = 1000
max_step = 200
gamma = 0.9
r = 0.99
lr = 0.001

## run
cartpole_env = Environment(num_episodes, max_step, gamma, r, lr)
cartpole_env.train()
cartpole_env.evaluate()
show_video(cartpole_env.env)

self.device =  cuda:0
0 Episode: Finished after 15 time steps with reward -86
1 Episode: Finished after 11 time steps with reward -90
2 Episode: Finished after 16 time steps with reward -85
3 Episode: Finished after 17 time steps with reward -84
4 Episode: Finished after 13 time steps with reward -88
5 Episode: Finished after 11 time steps with reward -90
6 Episode: Finished after 10 time steps with reward -91
7 Episode: Finished after 11 time steps with reward -90
8 Episode: Finished after 12 time steps with reward -89
9 Episode: Finished after 10 time steps with reward -91
10 Episode: Finished after 15 time steps with reward -86
11 Episode: Finished after 9 time steps with reward -92
12 Episode: Finished after 10 time steps with reward -91
13 Episode: Finished after 13 time steps with reward -88
14 Episode: Finished after 10 time steps with reward -91
15 Episode: Finished after 10 time steps with reward -91
16 Episode: Finished after 8 time steps with reward -93
17 Episode: Finished 

frame =  ('/content/videos/openaigym.video.1.60.video000001.mp4', '/content/videos/openaigym.video.1.60.video000001.meta.json')


frame =  ('/content/videos/openaigym.video.1.60.video000008.mp4', '/content/videos/openaigym.video.1.60.video000008.meta.json')


frame =  ('/content/videos/openaigym.video.1.60.video000027.mp4', '/content/videos/openaigym.video.1.60.video000027.meta.json')


frame =  ('/content/videos/openaigym.video.1.60.video000064.mp4', '/content/videos/openaigym.video.1.60.video000064.meta.json')


...

frame =  ('/content/videos/openaigym.video.1.60.video000216.mp4', '/content/videos/openaigym.video.1.60.video000216.meta.json')


frame =  ('/content/videos/openaigym.video.1.60.video000343.mp4', '/content/videos/openaigym.video.1.60.video000343.meta.json')


frame =  ('/content/videos/openaigym.video.1.60.video000512.mp4', '/content/videos/openaigym.video.1.60.video000512.meta.json')


frame =  ('/content/videos/openaigym.video.1.60.video000729.mp4', '/content/videos/openaigym.video.1.60.video000729.meta.json')


frame =  ('/content/videos/openaigym.video.1.60.video001000.mp4', '/content/videos/openaigym.video.1.60.video001000.meta.json')


## References
- [第15回　CartPole課題で深層強化学習DQNを実装](https://book.mynavi.jp/manatee/detail/id=89831)

## Appendix

In [ ]:
l = [
    ["a1", "a2", "a3"],
    ["b1", "b2", "b3"],
    ["c1", "c2", "c3"],
    ["d1", "d2", "d3"]
]
print("l = ", l)
print("*l = ", *l)
print("zip(*l) = ", zip(*l))
print("*zip(*l) = ", *zip(*l))

l =  [['a1', 'a2', 'a3'], ['b1', 'b2', 'b3'], ['c1', 'c2', 'c3'], ['d1', 'd2', 'd3']]
*l =  ['a1', 'a2', 'a3'] ['b1', 'b2', 'b3'] ['c1', 'c2', 'c3'] ['d1', 'd2', 'd3']
zip(*l) =  <zip object at 0x7f2436bd3870>
*zip(*l) =  ('a1', 'b1', 'c1', 'd1') ('a2', 'b2', 'c2', 'd2') ('a3', 'b3', 'c3', 'd3')


In [ ]:
import torch

input = torch.tensor([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
indices = torch.tensor([[0, 0, 0],[0, 1, 2],[2, 0, 1]])

print("input = \n", input)
print("indices = \n", indices)

## Switch values of the tensor according to indices of row (dim=0)
print("torch.gather(input=input, dim=0, index=indices) = \n", torch.gather(input=input, dim=0, index=indices))
'''
    input[0][0], input[0][1], input[0][2]
    input[0][0], input[1][1], input[2][2]
    input[2][0], input[0][1], input[1][2]
'''

## Switch values of the tensor according to indices of col (dim=1)
print("torch.gather(input=input, dim=1, index=indices) = \n", torch.gather(input=input, dim=1, index=indices))
'''
    input[0][0], input[0][0], input[0][0]
    input[1][0], input[1][1], input[2][2]
    input[2][2], input[2][0], input[2][1]
'''

input = 
 tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
indices = 
 tensor([[0, 0, 0],
        [0, 1, 2],
        [2, 0, 1]])
torch.gather(input=input, dim=0, index=indices) = 
 tensor([[1, 2, 3],
        [1, 5, 9],
        [7, 2, 6]])
torch.gather(input=input, dim=1, index=indices) = 
 tensor([[1, 1, 1],
        [4, 5, 6],
        [9, 7, 8]])
